In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork_dev as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn

# import models
from model_zoo import test_model

In [2]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/Users/juliankimura/Desktop/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf[:80000,:,:,:]
y_train = ltf[:80000,:]
X_valid = dtf_crossval[:1000,:,:,:]
y_valid = ltf_crossval[:1000,:]

CPU times: user 1.02 s, sys: 3.31 s, total: 4.34 s
Wall time: 4.46 s


In [3]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

In [4]:
model_layers, optimization = test_model.model(input_shape, num_labels)
nnbuild = nb.NeuralBuild(model_layers)
network, placeholders, hidden_feed_dict = nnbuild.get_network_build()

# build neural network class
nnmodel = nn.NeuralNet(network, placeholders, hidden_feed_dict)
nnmodel.inspect_layers()

# set output file paths

results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'test'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 32)
layer3: conv1_bias
(?, 2, 326, 32)
layer4: conv1_active
(?, 2, 326, 32)
layer5: conv1_dropout
(?, 2, 326, 32)
layer6: conv2
(?, 1, 322, 64)
layer7: conv2_bias
(?, 1, 322, 64)
layer8: conv2_active
(?, 1, 322, 64)
layer9: conv2_dropout
(?, 1, 322, 64)
layer10: conv3
(?, 1, 322, 128)
layer11: conv3_bias
(?, 1, 322, 128)
layer12: conv3_active
(?, 1, 322, 128)
layer13: conv3_dropout
(?, 1, 322, 128)
layer14: dense1
(?, 256)
layer15: dense1_bias
(?, 256)
layer16: dense1_active
(?, 256)
layer17: dense1_dropout
(?, 256)
layer18: dense2
(?, 2)
layer19: dense2_bias
(?, 2)
layer20: dense2_active
(?, 2)
layer21: output
(?, 2)
----------------------------------------------------------------------------


In [ ]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=200, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 
[==                            ] 8.0% -- time=307s -- loss=8.00858 -- accuracy=50.14%  

In [ ]:

batch_mean, batch_var = tf.nn.moments(, [0])

In [ ]:
x = tf.identity([1., 2.])

In [ ]:
ema_op = ema.apply(x)

In [ ]:
sess = tf.Session()
sess.run(ema)